In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

# Markov Decision Process

Die Grundlage für Reinforcement Learning bilden Markow-Entscheidungsprozesse. Aus diesem Grund sollen jene nun genauer betrachtet werden.
Ein Markov Decision Process (MDP) oder zu Deutsch auch Markow-Entscheidungsprozess ist ein Fünf-Tupel der Form $(S, A, T, R, s_0) $
wobei gilt:

* $S$ = eine Menge von Zuständen (states)
* $A$ = eine Menge von Aktionen (actions)
* $T(s, a, s’)$ = eine transition function mit $s \in S$ 
* $R$ = Belohnungsfunktion $R(s, a, s')$
* $s_0$ = Startzustand

Ein MDP ist immer von seinem Umfeld, dem Environment abhängig und kann optional einen Endzustand besitzen. Die Lösung von einem solchen Problem ist eine Funktion der Form $\pi \colon S\rightarrow A$. Diese wird auch Policy genannt. Die Policy gibt an, welche Aktion in welchem Zustand ausgeführt wird. Ziel ist es, eine (möglichst) optimale Policy zu finden. (vgl. Abbeel und Klein 2014 a)

$States$ sind eine Menge an Token, die angeben, in welchem Zustand sich jemand in der Umgebung befindet. Dies kann zum Beispiel die Position sein, in der sich jemand, bzw. genauer gesagt der Agent, in der Umgebung befindet. Ein $Agent$ kann in einer Umgebung ($Environment$ $e$) eine fest definierte Menge an $Aktionen$ ausführen, die ihn in einen neuen $State$ $s$ bringen.

Diese Theorie soll nun praktisch an einem Bespiel von einem Lieferwagen eingesetzt und erläutert werden, bevor näher auf die Transition- und Reward-Function eingegangen wird.

## Implementierung 1

### Das Problem
Wir sind Betreiber einer Spedition und sind zuständig für die Belieferung von Supermärkten. Dafür muss Ware zwischen Lagern und den Märkten innerhalb unserer Stadt transportiert werden. Unser innovativer LKW ist selbstfahrend und bekommt lediglich den Auftrag an einer Station (Lager oder Supermarkt) Ware einzusammeln und an einer anderen Station wieder abzuliefern.

### Definitionen
Wir bedienen zwei Lager und zwei Supermärkte.

Lager:
0. A-Lager (A)
1. B-Lager (B)

Supermarkt:
2. C-Markt (C)
3. D-Markt (D)

Diese verteilen sich folgendermaßen in unserer Stadt:

<img src="img/city.png" alt="stadt" width="400"/>

Die Stadt ist eine 6x6-Quadratestadt und mit 36 Positionen, die die Koordinaten $(0,0)$ bis $(5,5)$ haben, versehen. Der LKW kann sich frei in der Stadt bewegen, aber nicht durch die Grünstreifen fahren.

Die Anzahl der Zustände ($States$) ergibt sich folgendermaßen:
* 6 x 6 Positionen
* 4 Orte zu denen die Ware gebracht werden kann (A bis D bzw. 0 bis 3)
* 5 Orte, an denen sich die Ware befindet (A bis D bzw. 0 bis 3 und im LKW (Position Nr.4))

$$ 6 \cdot 6 \cdot 4 \cdot 5 = 720 \texttt{ mögliche Zustände}$$

Die Aktionen ($Actions$), die der LKW ausführen kann sind:
0. nach Norden fahren
1. nach Osten fahren
2. nach Süden fahren
3. nach Westen fahren
4. Ware einsammeln
5. Ware abladen

Dabei kann er folgende Belohnungen (und Abzüge) ($Rewards$) erhalten:
* Ware korrekt abliefern: +20
* Ware falsch einsammeln/abliefern: -10
* Pro Schritt: -1

### Definitionen implementieren

Zunächst werden alle Libraries, die jetzt oder im Verlaufe der gesamten Entwicklung benötigt werden importiert.

In [ ]:
import copy
import random
import time
import numpy as np

Im Folgenden werden die zuvor getroffenen Spezifikationen in ein für den Rechner verständliches Format überführt. Es gibt jeweils sechs Spalten und Reihen (0 bis 5) und eine Menge von Aktionen (ebenfalls 0 bis 5). Für die Ausgabe wird noch eine Überführung in eine Beschreibung vorgenommen. Außerdem werden die Koordinaten der Lager und Supermärkte festgehalten. Die Koordinaten haben dabei die Form `(Reihe, Spalte)`.

Die Grünstreifen werden in einer Menge gespeichert. Ein Grünstreifen liegt immer zwischen zwei Feldern. Diese werden in einem Tupel in der Reihenfolge `(Links, Rechts)` bzw. `(Oben, Unten)` angegeben. Daraus ergibt sich für jedes Stück eines Grünstreifens ein Tupel der Form `((Reihe Zelle links, Spalte Zelle links), (Reihe Zelle rechts, Spalte Zelle rechts))` bzw. mit "oben" und "unten", falls es sich um einen horizontalen Streifen handelt.

In [ ]:
rows = [row for row in range(0, 6)]
cols = [col for col in range(0, 6)]
actions = {action for action in range(0, 6)}
actions_description = ["Drive north", "Drive east", "Drive south", "Drive west", "Pickup goods", "Dropoff goods"]
stations = [(0,0), (0,3), (5,0), (4,5)]
stations_descriptions = ["Warehouse A", "Warehouse B", "Supermarket C", "Supermarket D"]
position_goods_descriptions = stations_descriptions + ["In the truck"]
walls = {
    ((0,2), (0,3)), #vertikal
    ((1,2), (1,3)),
    ((3,1), (3,2)),
    ((4,0), (4,1)),
    ((5,0), (5,1)),
    ((5,2), (5,3)),
    ((1,0), (2,0)), # horizontal
    ((1,5), (2,5)),
    ((3,4), (4,4)),
    ((3,5), (4,5))
}

Als nächstes soll für jedes Feld im Stadtplan festgehalten werden, welche Aktionen von diesem Feld ausgehend möglich sind. Zunächst wird allen Feldern alle Aktionen zugewiesen. Den am Rand liegenden Feldern wird die Aktion aberkannt, die aus der Stadt raus führen würde. Einsammel- und Ablieferaktionen sind nur an den zuvor definierten Stationen möglich, weshalb den anderen Feldern diese Aktion ebenfalls entzogen wird. Als nächstes werden Felder betrachtet, die in der direkten Nachbarschaft eines Grünstreifens liegen und dort die Aktionen entfernt, die den LKW dazu veranlassen würden, die Grünanlage zu zerstören. Daraus ergibt sich dann das gewünschte Dictionary mit dem Feld als Key und einer Menge möglicher Aktionen als Value.

In [ ]:
possible_actions = dict()

for row in rows:
    for col in cols:
        possible_actions[(row, col)] = copy.deepcopy(actions)
for key in possible_actions:
    (row, col) = key
    # Ränder
    if row == 0:
        possible_actions[key].remove(0)
    elif row == 5:
        possible_actions[key].remove(2)
    if col == 0:
        possible_actions[key].remove(3)
    elif col == 5:
        possible_actions[key].remove(1)
    # Einsammeln, Abliefern
    if (row, col) not in stations:
        possible_actions[key].remove(4)
        possible_actions[key].remove(5)
    # Grünstreifen
    if ((row, col), (row, col + 1)) in walls:
        possible_actions[key].remove(1)
    if ((row, col - 1), (row, col)) in walls:
        possible_actions[key].remove(3)
    if ((row, col), (row + 1, col)) in walls:
        possible_actions[key].remove(2)
    if ((row - 1, col), (row, col)) in walls:
        possible_actions[key].remove(0)

### Die Transport-Klasse

Die Transportklasse speichert den aktuellen Zustand (`state`), die grafische Darstellung (`canvas`) von diesem sowie den aktuell erreichten Wert (`current_value`). Außerdem speichert sie, wie viele Schritte bereits ausgeführt wurden (`steps`). Ebenfalls ist eine Methode zur Ermittlung der nächsten Aktion, die `action_method`, zu übergeben. Dies kann eine Methode sein, die eine zufällige Aktion zurückgibt (`random_action`) oder eine Methode, die die durch Q-Learning erlernten Werte ausnutzt (`q_learning_action`). Außerdem werden alle besuchten Zustände in `visited_states` aufgelistet. Ist die Variable `stepwise` auf `True` gesetzt, wird zur besseren Sichtbarkeit der Aktionen in der Darstellung nach jeder Aktion eine kurze Pause durchgeführt. Die graphische Darstellung erfolgt nur, wenn `visualize` auch `True` ist. Die Variable `done` markiert, ob der Transport der Waren erfolgreich abgeschlossen wurde. Für eine spätere Berechnung wird die Vorgängerposition der Waren benötigt, weshalb diese hier ebenfalls abgespeichert wird.

In [ ]:
class Transport():
    def __init__(self, state, action_method, stepwise = False, visualize = False):
        self.state = state
        self.canvas = self.init_canvas()
        self.action_method = action_method
        self.current_value = 0
        self.steps = 0
        self.visited_states = [self.state]
        self.state_list = []
        self.stepwise = stepwise
        self.visualize = visualize
        self.started = False
        self.done = False
        self.position_goods_old = self.state[1]

        self.canvas[3].on_mouse_down(self.handle_mouse_down)

## Transition-Funktion, Probability und Reward-Funktion

Alle Schritte, die bis jetzt beschrieben wurden, befassen sich mit dem Problem. Um eine Lösung zu finden, wird eine Policy-Funktion $\pi(s)$ aufgestellt, die für einen State $s$ eine Aktion $a$ zurückgibt, die die vermutlich höchste Belohnung für den Agenten bereithält. Die Policy-Funktion kennt Tripel der Form $<s,a,r>$ wobei $s$ der aktuelle State ist, $a$ die Aktion die im State $s$ ausgeführt wird und $r$ die Belohnung, die der Agent für die Aktion $a$ im State $s$ bekommen würde. Die Policy-Funktion hat viele von diesen Tripeln vorliegen und gibt die Aktion mit der höchsten Belohnung zurück. Eine optimale Policy $\pi*$ gibt immer die Aktion an, die langfristig die höchste Belohnung ergibt.

Der Prozess, also der Weg vom Start- zum Endzustand, hat dabei die Markov-Eigenschaft. Das bedeutet, dass der Folgezustand nur vom aktuellen Zustand abhängig ist und nicht auf den vorangegangenen basiert. Es gilt also:

$P(S_{t+1}=s_t'|s_t,a_t,s_{t-1},a_{t-1})  =  P(S_{t+1}=s_t'|s_t,a_t) \forall s \in S$

Das Markovsche Prinzip besagt, dass die vergangenen States oder Aktionen keinen Einfluss auf die aktuellen oder zukünftigen Entscheidungen haben. Die Wahrscheinlichkeit, die die Transition-Funktion berechnet ist unabhängig von den vorherigen States.

Die $Transition-Funktion$ gibt an, mit welcher Wahrscheinlichkeit man von einem Zustand in den nächsten gelangt. Es handelt sich also um eine Funktion der Art $P_{ss'}(s'| s, a)$. Dabei ist $P$ die Wahrscheinlichkeit, dass Aktion $a$ vom Zustand $s$ zu $s'$ führt. Das Ergebnis der Funktion kann mit Hilfe einer Matrix dargestellt werden:


$P = \begin{bmatrix}
P_{11} & ... & P_{1n}\\
... &  &\\
P_{n1} & ... & P_{nn}
\end{bmatrix}$ wobei $1$ bis $n$ alle möglichen Zustände bezeichnen

Die Summe der Werte aus einer Reihe der Matrix muss für alle Reihen 1 ergeben.

Für jeden Zustandswechsel kann eine Belohnung $r$ ("Reward") vergeben werden. Der Höhe der Belohnung ist durch die $Reward-Funktion$ festgelegt. In vielen MDPs wird die Belohnung mit einem Faktor $\gamma$, dem "Discount" multipliziert. Dabei ist $\gamma \in [0,1]$ und wird mit der Anzahl der bisher erfolgten Schritte $t$ potenziert. Der Discount-Faktor wird oftmals genutzt, um zu erreichen, das früher erhaltene Belohnungen stärker gewichtet werden als spätere. Der aktuelle Wert des Zustands ergibt sich aus der Summe der erhaltenen Belohnungen mal des Discounts. Ziel ist es, den Wert zu maximieren.
Es gilt:

$$\texttt{Wert} = \sum_{k = 0}^t \gamma^k\cdot r_{k+1}$$



Das Ganze soll nun wieder an unserem Beispiel erläutert werden.

## Implementierung 2

Die Transition function gibt für eine Aktion den Zustand zurück der folgt, wenn man auf den aktuellen Zustand die gewünschte Aktion anwendet. Dies ist möglich, da man aus der Kombination Zustand $s_t$ und Aktion $a$ in diesem Beispiel eindeutig den Folgezustand $s_{t+1}$ bestimmen kann, die Wahrscheinlichkeit für diesen Folgezustand also bei 100% liegt. Falls die Aktion nicht möglich ist, wird der alte Zustand zurückgegeben. Es gilt dann $s_t = s_{t+1}$. Ein Zustand ist dabei folgendermaßen aufgebaut:

`(Position LKW, Position Ware, Position Ziel)`.

Wobei gilt:
* Position LKW = (Reihe LKW, Spalte LKW)
* Position Ware $\in$ \[0, 4\] $\rightarrow$ \[Lager A, Lager B, Supermarkt C, Supermarkt D, im LKW\]
* Position Ziel $\in$ \[0, 3\] $\rightarrow$ \[Lager A, Lager B, Supermarkt C, Supermarkt D\]

Bei der Funktion handelt es sich um eine Funktion der Klasse `Transport`.

In [ ]:
def transition_function(self, action):
    position_truck, position_goods, position_goal = self.state
    if action not in possible_actions[position_truck]:
        return self.state
    
    row, col = position_truck
    if action == 0:
        position_truck = (row - 1, col)
    elif action == 1:
        position_truck = (row, col + 1) 
    elif action == 2:
        position_truck = (row + 1, col)
    elif action == 3:
        position_truck = (row, col - 1)
    elif action == 4:
        if position_goods == stations.index(position_truck):
            position_goods = 4
    elif action == 5:
        if position_goods != 4:
            return self.state
        position_goods = stations.index(position_truck)
            
    return (position_truck, position_goods, position_goal) 
Transport.transition_function = transition_function
del transition_function

`reward_function()` gibt die Belohnung zurück, die für die Aktion im aktuellen Zustand erzielt wird.
Bei der Funktion handelt es sich ebenfalls um eine Funktion der Klasse `Transport`.

In [ ]:
def reward_function(self, action):
    reward = -1 # Pro Schritt
    position_truck, position_goods, position_goal = self.state
    
    if action not in possible_actions[position_truck]: return reward
    
    if action == 4:
        station_truck = stations.index(position_truck)
        if position_goods != station_truck: # Ware falsch einsammeln
            reward -= 10
    elif action == 5:
        station_truck = stations.index(position_truck)
        if position_goal != station_truck and position_goods == 4: # Ware falsch abliefern  
            reward -= 10
        elif position_goal == station_truck and position_goods == 4: # Ware korrekt abliefern
            reward += 20
        else: # Abliefern ohne geladene Ware
            reward -= 10
            
    return reward
Transport.reward_function = reward_function
del reward_function